In [ ]:
''''In tis main file we do an initial explortation, we clean the data, and perfoirm EDA and vizualisation'''

import pandas as pd
import warnings

warnings.filterwarnings("ignore")

import initial_exploration as explo
import data_cleaning as cl
import eda as eda

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

# Data Quality Analysis (Data Cleaning)

### Cash Requests Analysis

In [ ]:
# Load datasets:
cash_requests = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

In [ ]:
# First look to cash_request data
cash_requests.head()

In [ ]:
# Inital exploration cash_request data
explo.check(cash_requests)

In [ ]:
# Cleaning the data for cash request data frame

# Formating dates column 
cl.convert_dates(cash_requests, cl.cash_request_date_columns)

# New look to the data after date columns fixed
cash_requests.head()

In [ ]:
# Let's ensure all the data is in the correct format
explo.check(cash_requests)

### Fees Analysis

In [ ]:
# First look to fees data
fees.head()

In [ ]:
# Inital exploration fees data
explo.check(fees)

In [ ]:
# Cleaning the data for fees data frame

# Formating dates column 
cl.convert_dates(fees, cl.fees_data_date_columns)

# New look to the data after date columns fixed
fees.head()

In [ ]:
# Let's ensure all the data is in the correct format
explo.check(fees)

### Data Cleaning 

In [ ]:
# Let's merge the two data frames. Data from cash request using the 'id' columns
# and fees data frame usiung the 'cash_request_id' column and keep cleaning

# Renaming column 'id' to 'cash_request_id' in cash request data frame for better understending
cash_requests = cl.rename_col(cash_requests, 'id', 'cash_request_id')

# Merging the data frames and saving them into a new data frame: data_df
data_df = cl.merge_df(cash_requests, fees, 'outer', 'cash_request_id')

# Renaming columns of equal name in both data frames. The columns from cash request to start with CR_
# and the once from fees to fee_
data_df = cl.rename_col_xy(data_df)

# Column with the fee id values is rename to id_fee for better understanding
data_df = cl.rename_col(data_df, 'id', 'id_fee')

# Reason column include the category + date or request id, resulting in many unique values
# with this modification we remove the date or request id and this way we ended with 4 unique 
# values that can be used for EDA
data_df = cl.clean_text_column(data_df, "reason")

# Removing the rows without a cash request amount
data_df = cl.remove_nan(data_df, "amount")

# Id fee column is not necesary for the analysis and category column have many missing values so we will drop
data_df = cl.drop_col(data_df, ["id_fee", "category"])

# Let's check this changes
data_df.head(20)

In [ ]:
# Exploration of the merged data frame
explo.check(data_df)

In [ ]:
# Creating individual data frames for columns that are numerical, categorical, and dates
frames = cl.selecting_data_types(data_df)
num = frames[1]
cat = frames[0]
date = frames[2]

In [ ]:
# First look to categorical columns
cat.head()

In [ ]:
# Total amount should be numerical so we are moving it from cat to num
cat, num = cl.move_column(cat, num, "total_amount")

#Let's check cat again
cat.head()

In [ ]:
# Setting CR created at as index for the cohort analysis 
cat = cl.set_index(cat, data_df, 'CR_created_at')

#Let's check cat again
cat.head()

In [ ]:
# First look to date data frame
date.head()

In [ ]:
# First look to categorical columns. User_id was kept for the cohort analysis
num.head()

In [ ]:
# Lets add more data to the numerical data frame using the dates column

# Time between a CR is created and Ironhack get the mnoney back
df_time_between_CR_and_money_back = cl.process_date_columns(date, "CR_created_at", "money_back_date", "days_difference_CR_back")
num = cl.merge_by_index(num, df_time_between_CR_and_money_back, "outer", "days_difference_CR_back")

# Time between a fee timestamp is created and Ironhack get the mnoney
df_time_between_fee_created_and_paid_at_date = cl.process_date_columns(date, "fee_created_at", "paid_at", "days_difference_fee_paid")
num = cl.merge_by_index(num, df_time_between_fee_created_and_paid_at_date, "outer", "days_difference_fee_paid")


# A new look to numerical data frame
num.head(10)

In [ ]:
# Setting CR created at as index for the cohort analysis 
num = cl.set_index(num, data_df, 'CR_created_at')

# A new look to numerical data frame
num.head(10)

# Initial Exploratory Data Analysis (EDA)

### Categorical columns EDA

In [ ]:
# Generate frequency table and bar plots for category columns
for column in cat.columns:
    print(column)
    df0, df1 = eda.freq(cat, column)
    print(df0)
    eda.cat_viz(cat, df1, column)
    print("")

### Numerical columns EDA

In [ ]:
# EDA for numerical columns
for column in num.columns:
    print(column)
    print(eda.stat(num, column))
    eda.stat_viz(num,column)
    print("")

# Cohort Analysis

## Univariate

### Frequency

In [ ]:
# Copy of data_df
data_fre = data_df.copy()

# Frequency hit mpa by cohort
cohort_size, cohort_table = eda.fre(data_fre)
cohort_size

In [ ]:
# Frequency of service usage per month
frequency_table = eda.fre_bar(cohort_table)
frequency_table

### Incident Rate

In [ ]:
#Copy of categorical data frame
cat_r = data_df.copy()

# Incident Rate: The incident rate, specifically focusing on payment incidents, for each cohort
incident_rate, frequency_table = eda.incident_metric(cat_r)

# Let's have a look to the incident rate table
incident_rate

In [ ]:
# Incident Rate: The incident rate, specifically focusing on payment incidents, for each cohort
frequency = eda.incident_metric_f(cat_r, frequency_table)

# Let's have a look to the incident rate table
frequency

### Revenue Generated by the Cohort

In [ ]:
#Copy of numerical data frame
num_r = num.copy()

# Revenue Generated by the Cohort: Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.
revenue = eda.revenue_metric(num_r, "total_amount")

# Let's have a look to the revenue table
revenue

In [ ]:
# Let's plot the total revenue per month
eda.revenue_plot(revenue)

In [ ]:
cohort_data = eda.revenue_plot_per_user(num_r)

## Bivariate

In [ ]:
#Let's make a copy of the data for the bivariate analysis
cat_new = cat.copy()
num_new = cohort_data.copy()

# Corelation between the diferent incidents and the revenue  
df_merged, df_melted = eda.bi_rev_incid(cat_new, num_new)
df_merged

In [ ]:
# Lets restructure the data
cat_new = cat.copy()
num_new = num_r.copy()
new_data_str = eda.restruct(cat_new, num_new)

In [ ]:
# Data restructured 1
new_data_str[0]

In [ ]:
# Data restructured 2
new_data_str[1]

In [ ]:
# Data restructured 1
new_data_str[2]

In [ ]:
# Study of the correlation between a fee incident and status of the fee
crosstab_table = eda.fee_heat_map(data_df, "fee_status", "reason", "Fee reason", "Fee Status")
crosstab_table

In [ ]:
# Study of the correlation between a transfer type and cash request status
crosstab_table = eda.fee_heat_map(data_df, "CR_status", "transfer_type", "Transfer type", "Cash request status")
crosstab_table

In [ ]:
# Study of the correlation between a fee reason and days that take to recibe the payment
eda.cat_con(cat, num)


In [ ]:
# Study of the correlation between the time for a CR payment and a Fee payment
eda.line_plot(cat, num)